In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss

In [2]:
#function for reading database, return a dataframe, return a dataframe
def readDB(csv_database):
    sql = 'select * from sensorProcessed' #sql need to be executed. 
    df = pd.read_sql_query(sql, csv_database) # read data as dataframe
    return df

In [3]:
#set credentials file for plotly(username & API key)
import plotly
username = 'mengyizhou95' #username registered on Ploty
api_key = 'X5DOiJATuUqPbsabrbxA' #the api key plotly asigned
plotly.tools.set_credentials_file(username=username, api_key=api_key)

In [4]:
#set the conversion ratio from min to ms
timeConversionVal = 60000

In [5]:
#function for trans time from min to ms
def MinToMS(time, df):
    time = time * timeConversionVal
    time = df.time.iloc[(df['time']-time).abs().argsort()[:1]] #find the closest time on table
    return time.values[0]

In [1]:
#trim data function for 23-col data
def trimData_23(df, start, end, filename='trimedData.csv'):
    #trans time
    start = MinToMS(start, df)
    end = MinToMS(end, df)
    #get index
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #make time start from 0
    df = df.iloc[startIndex:endIndex]
    tp = df.iloc[0]['time'].item()
    df["time"] = df["time"].apply(lambda x: x - tp)
    #reset index number
    df = df.drop(columns=['index'])
    df = df.reset_index(drop=True)
    #rename col
    df.columns = [
                  'time', \
                  'flapper0', 'flapper1', 'flapper2', 'flapper3', 'flapper4', 'flapper5', 'flapper6', 'flapper7', \
                  'bend', 'light', \
                  'accX', 'accY', 'accZ', \
                  'compassX', 'compassY', 'compassZ', \
                  'rotationalSpeedX', 'rotationalSpeedY', 'rotationalSpeedZ', \
                  'orientationX', 'orientationY', 'orientationZ']
    
    df.to_csv(filename)

In [2]:
#if we stop in the middle, we can concat two parts of data 
def concatData(df1, start1, end1, start2, end2, filename='concatData.csv'):
    #trans time
    start1 = MinToMS(start1, df1)
    end1 = MinToMS(end1, df1)
    start2 = MinToMS(start2, df1)
    end2 = MinToMS(end2, df1)
    
    #get index
    startIndex1 = df1.index[df1['time'] == start1].tolist()[0]
    endIndex1 = df1.index[df1['time'] == end1].tolist()[0]
    startIndex2 = df1.index[df1['time'] == start2].tolist()[0]
    endIndex2 = df1.index[df1['time'] == end2].tolist()[0]
    
    #get two dataframe to concat
    dftmp1 = df1.iloc[startIndex1:endIndex1]
    dftmp2 = df1.iloc[startIndex2:endIndex2]
    
    #concat two parts
    diff = start2 - end1
    dftmp1 = dftmp1.reset_index(drop=True)
    dftmp2 = dftmp2.reset_index(drop=True)
    dftmp2["time"] = dftmp2["time"].apply(lambda x: x - diff)

    df1 = pd.concat([dftmp1, dftmp2])
    df1 = df1.reset_index(drop=True)
    
    #make time start from 0
    first = df1['time'].head(1).tolist()[0]
    df1['time'] = df1['time'] - first
    
    #to csv
    df1.to_csv("Cambridge_trim.csv")